# Bank Customer Segmentation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('bank_transactions.csv')

In [3]:
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


In [5]:
data.describe()

,CustAccountBalance,TransactionTime,TransactionAmount (INR)
count,1.046198e+06,1.048567e+06,1.048567e+06
mean,1.154035e+05,1.570875e+05,1.574335e+03
std,8.464854e+05,5.126185e+04,6.574743e+03
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.721760e+03,1.240300e+05,1.610000e+02
50%,1.679218e+04,1.642260e+05,4.590300e+02
75%,5.765736e+04,2.000100e+05,1.200000e+03
max,1.150355e+08,2.359590e+05,1.560035e+06


In [6]:
data.isna().sum()

TransactionID                 0
CustomerID                    0
CustomerDOB                3397
CustGender                 1100
CustLocation                151
CustAccountBalance         2369
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

### Preprocessing

In [7]:
# parse dates
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'], dayfirst=True, errors='coerce')  # your data is d/m/yy
# TransactionTime looks like HHMMSS or HHMM e.g., 143207 -> convert to time
def parse_time(t):
    try:
        s = str(int(t)).zfill(6)   # ensure 6 digits
        return pd.to_datetime(s, format='%H%M%S').time()
    except:
        return pd.NaT

data['TransactionTime'] = data['TransactionTime'].apply(parse_time)
data['TransactionDateTime'] = pd.to_datetime(data['TransactionDate'].dt.date.astype(str) + ' ' + data['TransactionTime'].astype(str),
                                           errors='coerce')

# DOB -> age
data['CustomerDOB'] = pd.to_datetime(data['CustomerDOB'], dayfirst=True, errors='coerce')
# compute age at transaction
data['age'] = ((data['TransactionDate'] - data['CustomerDOB']).dt.days / 365.25).astype('float')
# fill implausible ages
data.loc[data['age'] < 10, 'age'] = np.nan
data.loc[data['age'] > 120, 'age'] = np.nan

# basic cleaning for categorical
data['CustGender'] = data['CustGender'].fillna('U').str.upper().replace({'F':'F','M':'M','U':'U'})
data['CustLocation'] = data['CustLocation'].str.upper().fillna('UNKNOWN')

# numeric fills
data['CustAccountBalance'] = data['CustAccountBalance'].fillna(data['CustAccountBalance'].median())

# transform amount: add small constant and log transform to reduce skew
data['txn_amount'] = data['TransactionAmount (INR)'].astype(float)
data['txn_amount_log'] = np.log1p(data['txn_amount'])

# time-of-day and weekday features
data['hour'] = data['TransactionDateTime'].dt.hour.fillna(-1).astype(int)
data['weekday'] = data['TransactionDateTime'].dt.weekday.fillna(-1).astype(int)


C:\Users\meetb\AppData\Local\Temp\ipykernel_42516\133516007.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['TransactionDate'] = pd.to_datetime(data['TransactionDate'], dayfirst=True, errors='coerce')  # your data is d/m/yy
C:\Users\meetb\AppData\Local\Temp\ipykernel_42516\133516007.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['CustomerDOB'] = pd.to_datetime(data['CustomerDOB'], dayfirst=True, errors='coerce')


### 1. Performing clustering on the dataset to identify popular customer groups.

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# aggregate per customer
cust = data.groupby('CustomerID').agg(
    txn_count = ('txn_amount', 'count'),
    txn_sum = ('txn_amount', 'sum'),
    txn_mean = ('txn_amount', 'mean'),
    txn_median = ('txn_amount', 'median'),
    txn_std = ('txn_amount', 'std'),
    last_txn = ('TransactionDate', 'max'),
    first_txn = ('TransactionDate', 'min'),
    avg_balance = ('CustAccountBalance', 'mean'),
    age = ('age', 'median')
).reset_index()

# compute recency (days since last txn) relative to global last date
reference_date = data['TransactionDate'].max() + pd.Timedelta(days=1)
cust['recency_days'] = (reference_date - cust['last_txn']).dt.days
cust['active_days'] = (cust['last_txn'] - cust['first_txn']).dt.days.replace(0,1)
cust['txn_freq_per_day'] = cust['txn_count'] / cust['active_days']

# replace nans
cust['txn_std'] = cust['txn_std'].fillna(0)
cust['age'] = cust['age'].fillna(cust['age'].median())

# features for clustering
features = ['txn_count','txn_sum','txn_mean','txn_median','txn_std','avg_balance','recency_days','txn_freq_per_day','age']
X = cust[features].copy()
X = X.fillna(0)

# scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# elbow and silhouette
inertia = []
sil_scores = []
K = range(2,13)
for k in K:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    inertia.append(km.inertia_)
    sil_scores.append(silhouette_score(X_scaled, labels))

# plot
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(K, inertia, '-o')
plt.xlabel('k'); plt.ylabel('Inertia (sum of squared distances)'); plt.title('Elbow method')

plt.subplot(1,2,2)
plt.plot(K, sil_scores, '-o')
plt.xlabel('k'); plt.ylabel('Silhouette Score'); plt.title('Silhouette method')
plt.tight_layout()
plt.show()


KeyboardInterrupt: 

### 2. Location wise Analysis

In [ ]:
# aggregate by location (assume CustLocation is state or city; prefer state)
loc = data.groupby('CustLocation').agg(
    total_volume=('txn_amount','sum'),
    txn_count=('txn_amount','count'),
    avg_txn=('txn_amount','mean'),
    median_balance=('CustAccountBalance','median'),
    unique_customers=('CustomerID','nunique')
).reset_index()

loc['avg_txn_per_customer'] = loc['total_volume'] / loc['unique_customers']
loc = loc.sort_values('total_volume', ascending=False).head(50)
print(loc.head(20))
